In [1]:
import sys
from IPython.display import display

sys.path.append('../scripts')  
from models import *
from df_handling import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import warnings
import statistics
warnings.filterwarnings("ignore")


In [2]:
#load training datasets 
train1 = pd.read_csv('../data/fullfacts_emb.csv')
train2 = pd.read_csv('../data/greek_web_scraping_emb.csv')
train2_1 = pd.read_csv('../data/greek_web_scraping.csv')
train3 = pd.read_csv('../data/snopes2_emb.csv', nrows=1000)
train4 = pd.read_csv('../data/reuters_emb.csv')
train5  = pd.read_csv('../data/test1.csv')
train6 = pd.read_csv('../data/cyprus_factcheck_emb.csv')

train1 = unravel_df(train1, ['text_embedding'])
train2 = unravel_df(train2, ['text_embedding'])
train3 = unravel_df(train3, ['text_embedding'])
train4 = unravel_df(train4, ['text_embedding'])
train5 = unravel_df(train5, ['text_embedding'])
train6 = unravel_df(train6, ['text_embedding'])
train5.rename(columns={'text': 'claim', 'predict_label': 'label'}, inplace=True)


#find whick grek data is from greekhoaxes etc
train2_1 = train2_1[['id','url']]
train2_1['source'] = train2_1['url'].apply(lambda x: 0 if "ellinikahoaxes" in x else (1 if "factcheckgreek" in x else None))
train2 = pd.merge(train2, train2_1[['id','source']], on='id', how='inner')

In [3]:
train1['source'] = 2
train3['source'] = 3
train4['source'] = 4
train5['source'] = 2 #check4facts
train6['source'] = 3 #cyprus



In [4]:
df_train = pd.concat([train2,train5,train6], ignore_index=True)


In [20]:
train2.shape[0] + train5.shape[0] + train6.shape[0]

803

In [6]:
NaiveBayes_list  = list()
LR_list  = list()
KNeigh_list  = list()
RF_list  = list()
DT_list  = list()
SVC_list  = list()
MLP_list  = list()
data_source_train_list = list()
data_source_val_list = list()

#list for sub accuracies
NaiveBayes_list_acc  = list()
LR_list_acc  = list()
KNeigh_list_acc  = list()
RF_list_acc  = list()
DT_list_acc  = list()
SVC_list_acc  = list()
MLP_list_acc  = list()


In [7]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for fold, (train_index, valid_index) in enumerate(kf.split(df_train.text_embedding), 1):

    X_train, X_valid = [df_train.text_embedding[i] for i in train_index], [df_train.text_embedding[i] for i in valid_index ]
    y_train, y_valid = df_train.label[train_index] , df_train.label[valid_index]
    data_source_train, data_source_val = df_train.source[train_index], df_train.source[valid_index]
    

    print(f"Fold {fold}:")
    print("Training data:\n ")
    print(data_source_train.value_counts(), '\n')
    print("Validation data:\n ")
    print(data_source_val.value_counts())
    print("---------------------")

    data_source_train_list.append(data_source_train.value_counts())
    data_source_val_list.append(data_source_val.value_counts())

   
    
    

    nb1, nb2 = cross_val_Naive_Bayes(X_train, y_train, X_valid, y_valid, data_source_val)
    lr1, lr2 = cross_val_LR(X_train, y_train, X_valid, y_valid, data_source_val)
    kn1, kn2 = cross_val_Kneigh(X_train, y_train, X_valid, y_valid, data_source_val)
    svm1, svm2 = cross_val_SVC(X_train, y_train, X_valid, y_valid, data_source_val)
    rf1, rf2 = cross_val_RF(X_train, y_train, X_valid, y_valid, data_source_val)
    dt1, dt2 = cross_val_DT(X_train, y_train, X_valid, y_valid, data_source_val)
    mlp1, mlp2 = cross_val_MLP(X_train, y_train, X_valid, y_valid, data_source_val)

    NaiveBayes_list.append(nb1)
    NaiveBayes_list_acc.append(nb2)

    LR_list.append(lr1)
    LR_list_acc.append(lr2)
 
    KNeigh_list.append(kn1)
    KNeigh_list_acc.append(kn2)

    SVC_list.append(svm1)
    SVC_list_acc.append(svm2)

    RF_list.append(rf1)
    RF_list_acc.append(rf2)

    DT_list.append(dt1)
    DT_list_acc.append(dt2)

    MLP_list.append(mlp1)
    MLP_list_acc.append(mlp2)
    


    


    


Fold 1:
Training data:
 
1    283
0    192
2    130
3    117
Name: source, dtype: int64 

Validation data:
 
0    27
1    24
2    18
3    12
Name: source, dtype: int64
---------------------
[23, 30, 31, 33, 39, 49, 63, 65, 66, 67, 78, 86, 96, 109, 120, 137, 139, 155, 168, 174, 192, 196, 199, 208, 209, 210, 218]

Accuracy for data source 0: 0.9259259259259259
[227, 231, 235, 266, 275, 294, 299, 300, 311, 323, 326, 331, 342, 367, 393, 394, 395, 405, 423, 434, 467, 513, 521, 523]

Accuracy for data source 1: 0.9583333333333334
[527, 528, 529, 535, 547, 558, 567, 591, 595, 596, 597, 602, 605, 638, 653, 658, 669, 670]

Accuracy for data source 2: 0.7222222222222222
[692, 704, 723, 738, 741, 751, 758, 763, 780, 789, 798, 799]

Accuracy for data source 3: 0.6666666666666666
Accuracy for Naive Bayes: 0.8518518518518519

[(0, 0.9259259259259259), (1, 0.9583333333333334), (2, 0.7222222222222222), (3, 0.6666666666666666)]

Accuracy for data source 0: 0.9259259259259259

Accuracy for data source 1

  File "c:\Users\johnk\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



Accuracy for data source 0: 0.8518518518518519

Accuracy for data source 1: 1.0

Accuracy for data source 2: 0.5555555555555556

Accuracy for data source 3: 0.8333333333333334
Accuracy for KNeigh: 0.8271604938271605

[(0, 0.8518518518518519), (1, 1.0), (2, 0.5555555555555556), (3, 0.8333333333333334)]

Accuracy for data source 0: 0.9629629629629629

Accuracy for data source 1: 1.0

Accuracy for data source 2: 0.6666666666666666

Accuracy for data source 3: 0.6666666666666666
Accuracy for SVM: 0.8641975308641975

[(0, 0.9629629629629629), (1, 1.0), (2, 0.6666666666666666), (3, 0.6666666666666666)]

Accuracy for data source 0: 0.9259259259259259

Accuracy for data source 1: 0.9583333333333334

Accuracy for data source 2: 0.6666666666666666

Accuracy for data source 3: 0.9166666666666666
Accuracy for Random Forest: 0.8765432098765432

[(0, 0.9259259259259259), (1, 0.9583333333333334), (2, 0.6666666666666666), (3, 0.9166666666666666)]

Accuracy for data source 0: 0.6666666666666666

Accur

In [8]:
print(statistics.stdev(NaiveBayes_list))
print("LR: ", statistics.stdev(LR_list))
print("k-Neigh: ", statistics.stdev(KNeigh_list))
print("SVC: ", statistics.stdev(SVC_list))
print("RF: ", statistics.stdev(RF_list))
print("DT: ", statistics.stdev(DT_list))
print("MLP: ", statistics.stdev(MLP_list))

0.03604865822810527
LR:  0.026164176726240815
k-Neigh:  0.03989228540871212
SVC:  0.03254939825968141
RF:  0.028136496457354356
DT:  0.029345271872769016
MLP:  0.04201612586171298


In [9]:
data_source_train_list

[1    283
 0    192
 2    130
 3    117
 Name: source, dtype: int64,
 1    271
 0    200
 2    135
 3    116
 Name: source, dtype: int64,
 1    269
 0    195
 2    134
 3    124
 Name: source, dtype: int64,
 1    281
 0    196
 2    132
 3    114
 Name: source, dtype: int64,
 1    274
 0    201
 2    134
 3    114
 Name: source, dtype: int64,
 1    274
 0    194
 2    134
 3    121
 Name: source, dtype: int64,
 1    286
 0    197
 2    132
 3    108
 Name: source, dtype: int64,
 1    270
 0    206
 2    132
 3    115
 Name: source, dtype: int64,
 1    282
 0    195
 2    133
 3    113
 Name: source, dtype: int64,
 1    273
 0    195
 2    136
 3    119
 Name: source, dtype: int64]

In [10]:
df_corr = pd.DataFrame( {'factcheck_train_count':[data_source_train_list[i][1] for i in range (len(data_source_train_list))], 
                        'greekhoaxes_train_count': [data_source_train_list[i][0] for i in range (len(data_source_train_list))],
                        'check4facts_train_count': [data_source_train_list[i][2] for i in range (len(data_source_train_list))],
                        'cyprus_train_count': [data_source_train_list[i][3] for i in range (len(data_source_train_list))],
                        'check4facts_val_count':[data_source_val_list[i][2] for i in range (len(data_source_val_list))],
                        'factcheck_val_count':[data_source_val_list[i][1] for i in range (len(data_source_val_list))], 
                        'greekhoaxes_val_count': [data_source_val_list[i][0] for i in range (len(data_source_val_list))],
                        'cyprus_val_count': [data_source_val_list[i][3] for i in range (len(data_source_val_list))],


                        'NB_greekhoaxes_acc': [t[0][1] for t in NaiveBayes_list_acc], 'NB_factcheck_acc': [t[1][1] for t in NaiveBayes_list_acc],'NB_check4facts_acc': [t[2][1] for t in NaiveBayes_list_acc],
                        'LR_greekhoaxes_acc': [t[0][1] for t in LR_list_acc], 'lr_factcheck_acc': [t[1][1] for t in LR_list_acc],'lr_check4facts_acc': [t[2][1] for t in LR_list_acc],
                        'kn_greekhoaxes_acc': [t[0][1] for t in KNeigh_list_acc], 'kn_factcheck_acc': [t[1][1] for t in KNeigh_list_acc],'kn_check4facts_acc': [t[2][1] for t in KNeigh_list_acc],
                        'rf_greekhoaxes_acc': [t[0][1] for t in RF_list_acc], 'rf_factcheck_acc': [t[1][1] for t in RF_list_acc],'rf_check4facts_acc': [t[2][1] for t in RF_list_acc],
                        'dt_greeekhoaxes_acc': [t[0][1] for t in DT_list_acc], 'dt_factcheck_acc': [t[1][1] for t in DT_list_acc],'dt_check4facts_acc': [t[2][1] for t in DT_list_acc],
                        'svc_greekhoaxes_acc': [t[0][1] for t in SVC_list_acc], 'svc_factcheck_acc': [t[1][1] for t in SVC_list_acc],'svc_check4facts_acc': [t[2][1] for t in SVC_list_acc],
                        'mlp_greekhoaxes_acc': [t[0][1] for t in MLP_list_acc], 'mlp_factcheck_acc': [t[1][1] for t in MLP_list_acc],'mlp_check4facts_acc': [t[2][1] for t in MLP_list_acc],'mlp_cyprus_acc': [t[3][1] for t in MLP_list_acc],
                         






                        'NB_accuracy': NaiveBayes_list,'LR_accuracy': LR_list,'kn_accuracy': KNeigh_list,'RF_accuracy': RF_list,
                        'DT_accuracy': DT_list,'SVC_accuracy': SVC_list,'MLP_accuracy': MLP_list})






models = ['NB_', 'LR_', 'KN_', 'svc_', 'dt_','rf_','mlp_']
data = ['train','val']
sources = ['greekhoaxes_','factcheck_','check4facts_', 'cyprus_']

for s in sources:
    for d in data:
        column = s + d + '_per'
        df_corr[column] = df_corr[s+d+'_count']/(sum(df_corr[s+d+'_count'] for s in sources))
       

In [11]:
sources = ['greekhoaxes_','factcheck_','check4facts_', 'cyprus_']
for s in sources:
    res = pd.DataFrame(df_corr[['MLP_accuracy', s+'train_per',s+'val_per', 'mlp_'+s+'acc']])
    display(res)

,MLP_accuracy,greekhoaxes_train_per,greekhoaxes_val_per,mlp_greekhoaxes_acc
0,0.827160,0.265928,0.333333,0.851852
1,0.876543,0.277008,0.234568,0.894737
2,0.802469,0.270083,0.296296,0.875000
3,0.787500,0.271093,0.287500,0.739130
4,0.850000,0.278008,0.225000,1.000000
5,0.825000,0.268326,0.312500,0.880000
6,0.862500,0.272476,0.275000,0.954545
7,0.900000,0.284924,0.162500,0.923077
8,0.887500,0.269710,0.300000,1.000000
9,0.912500,0.269710,0.300000,1.000000


,MLP_accuracy,factcheck_train_per,factcheck_val_per,mlp_factcheck_acc
0,0.827160,0.391967,0.296296,0.958333
1,0.876543,0.375346,0.444444,0.972222
2,0.802469,0.372576,0.469136,0.921053
3,0.787500,0.388658,0.325000,1.000000
4,0.850000,0.378976,0.412500,0.969697
5,0.825000,0.378976,0.412500,0.939394
6,0.862500,0.395574,0.262500,1.000000
7,0.900000,0.373444,0.462500,0.972973
8,0.887500,0.390041,0.312500,1.000000
9,0.912500,0.377593,0.425000,0.941176


,MLP_accuracy,check4facts_train_per,check4facts_val_per,mlp_check4facts_acc
0,0.827160,0.180055,0.222222,0.555556
1,0.876543,0.186981,0.160494,0.461538
2,0.802469,0.185596,0.172840,0.357143
3,0.787500,0.182573,0.200000,0.375000
4,0.850000,0.185339,0.175000,0.357143
5,0.825000,0.185339,0.175000,0.428571
6,0.862500,0.182573,0.200000,0.375000
7,0.900000,0.182573,0.200000,0.625000
8,0.887500,0.183956,0.187500,0.400000
9,0.912500,0.188105,0.150000,0.666667


,MLP_accuracy,cyprus_train_per,cyprus_val_per,mlp_cyprus_acc
0,0.827160,0.162050,0.148148,0.916667
1,0.876543,0.160665,0.160494,1.000000
2,0.802469,0.171745,0.061728,0.800000
3,0.787500,0.157676,0.187500,0.933333
4,0.850000,0.157676,0.187500,0.866667
5,0.825000,0.167358,0.100000,0.875000
6,0.862500,0.149378,0.262500,1.000000
7,0.900000,0.159059,0.175000,1.000000
8,0.887500,0.156293,0.200000,1.000000
9,0.912500,0.164592,0.125000,0.900000


In [13]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# correlation_matrix = df_corr.corr()

# for i in range(8,51,13):
#     # Plot the correlation matrix using Seaborn
#     plt.figure(figsize=(30, 8))
#     sns.heatmap(correlation_matrix.iloc[:8, i:i+13 ], annot=True, cmap='coolwarm', fmt=".4f")
#     plt.title('Correlation Matrix')
#     plt.show()
#     print(i)

In [ ]:
for t in RF_list_acc:
    print(t)

[(0, 0.9545454545454546), (1, 0.9615384615384616), (2, 0.45)]
[(0, 0.8235294117647058), (1, 1.0), (2, 0.5)]
[(0, 0.75), (1, 0.9696969696969697), (2, 0.6)]
[(0, 1.0), (1, 1.0), (2, 0.6153846153846154)]
[(0, 1.0), (1, 1.0), (2, 0.5882352941176471)]
[(0, 0.9473684210526315), (1, 0.8787878787878788), (2, 0.26666666666666666)]
[(0, 0.9444444444444444), (1, 0.9142857142857143), (2, 0.6428571428571429)]
[(0, 0.9230769230769231), (1, 0.9487179487179487), (2, 0.6)]
[(0, 0.9615384615384616), (1, 0.92), (2, 0.625)]
[(0, 0.9545454545454546), (1, 0.8823529411764706), (2, 0.45454545454545453)]


In [ ]:
RF_list_acc

[[(0, 0.9545454545454546), (1, 0.9615384615384616), (2, 0.45)],
 [(0, 0.8235294117647058), (1, 1.0), (2, 0.5)],
 [(0, 0.75), (1, 0.9696969696969697), (2, 0.6)],
 [(0, 1.0), (1, 1.0), (2, 0.6153846153846154)],
 [(0, 1.0), (1, 1.0), (2, 0.5882352941176471)],
 [(0, 0.9473684210526315), (1, 0.8787878787878788), (2, 0.26666666666666666)],
 [(0, 0.9444444444444444), (1, 0.9142857142857143), (2, 0.6428571428571429)],
 [(0, 0.9230769230769231), (1, 0.9487179487179487), (2, 0.6)],
 [(0, 0.9615384615384616), (1, 0.92), (2, 0.625)],
 [(0, 0.9545454545454546), (1, 0.8823529411764706), (2, 0.45454545454545453)]]